In [85]:
from rdflib.util import get_tree, find_roots
from rdflib.term import *
from rdflib import Graph
from itertools import chain
from functools import reduce
import copy

# def handle 


def queryToGrammar(q):
    g = Graph()
    gr = Grammar(Pypher())
    treebuffer = []
    
    class Tree():
        def __init__(self):
            self.tree = {}
            self.variables = ['x', 'y', 'z']
            self.varptr = 0
            
        def addToTree(self, func):
            var = self.variables[self.varptr]
            self.tree[var] = func
        
    askquery = True if q[1].name == 'AskQuery' else False
    
    triples = q[1]['where']['part'][0]['triples']
    triples = [(i[0], i[1]['part'][0]['part'][0]['part'], i[2]) for i in triples]
    
    for triple in triples:
        g.add(triple)
        
    if askquery:
        triple = triples[0]
        out = "in({}, find({}, {}))".format(triple[0], triple[2], triple[1])
    else:
        select = 'uri'
        
        triples_without_select_var = copy.deepcopy(g)
        triples_without_select_var -= g.triples((Variable(select), None, None))
        triples_without_select_var -= g.triples((None, None, Variable(select)))

        non_select_entities_in_where = list(triples_without_select_var.subjects()) + list(triples_without_select_var.objects())
        x_count = non_select_entities_in_where.count(True)
        x_finds = []
        x_finds_raw = []

        if non_select_entities_in_where:
            for s,p,o in triples_without_select_var:
                if s == Variable('x'):
                    x_finds_raw += [(o, p)] 
                    x_finds.append("find({}, {})".format(o, p))
                if o == Variable('x'):
                    x_finds_raw += [(s, f"-{p}")] 
                    x_finds.append("find({}, reverse({}))".format(s, p))

            assert len(x_finds) in [1,2]

            if len(x_finds) == 1:
                x_finds = x_finds[0]
            else:
                x_finds = reduce(lambda a,b: "intersection({}, {})".format(a, b), x_finds)

                
                
        select_finds = []
        select_g = Graph()
        select_g += g.triples((Variable(select), None, None))
        select_g += g.triples((None, None, Variable(select)))

        tree = {'x': [], 'uri': []}

        for s,p,o in select_g:
            if Variable('x') not in [s, o]:
                if s == Variable(select):
                    tree['uri'] += [(o, p)]
                    select_finds.append("find({}, {})".format(o, p))
                if o == Variable(select):
                    tree['uri'] += [(s, f"-{p}")]
                    select_finds.append("find({}, reverse({}))".format(s, p))
            else:
                if s == Variable('x'):
                    tree['x'] = x_finds_raw
                    tree['uri'] += [('x', f"-{p}")]
                    select_finds.append("find({}, reverse({}))".format(x_finds, p))
                if o == Variable('x'):
                    tree['x'] = x_finds_raw
                    tree['uri'] += [('x', f"-{p}")]
                    select_finds.append("find({}, {})".format(x_finds, p))

        if len(select_finds) == 1:
            select_finds = select_finds[0]
        else:
            select_finds = reduce(lambda a,b: "intersection({}, {})".format(a, b), select_finds)


        for i, elements in enumerate(tree.items()):
            var, er = elements
            for entity, relation in er:
                print(elements)
                gr.find(var, f'o{i}', entity, relation)
            
        print(tree)
        out = select_finds
        out2 = gr.get_return('uri')
    
    return out, out2




# Testing queryToGrammar
    
from rdflib.plugins.sparql.parser import parseQuery
from rdflib.plugins.sparql.parserutils import prettify_parsetree
from rdflib.term import *

import json, codecs
with codecs.open("/home/nilesh/python/lcquad/templates.json") as fp:
    temps = json.load(fp)
    temps = {x["id"]: x["template"] for x in temps}

for idx, sq in list(temps.items()):
    if idx != 309:
        continue
    try:
        q = parseQuery(sq)
    #     print(prettify_parsetree(q))
        print(idx)
        out, out2 = queryToGrammar(q)
        print(sq)
        print(out)
        print(out2)
        print(out2.bound_params)
#     try:
#         print(queryToGrammar(q))
    except Exception as e:
        print (repr(e))
        print("ERROR:", idx, q)
        



309
('x', [(rdflib.term.URIRef('class'), rdflib.term.URIRef('rdf:type')), (rdflib.term.URIRef('%(e_in_in)s'), '-%(e_in_in_to_e_in)s')])
('x', [(rdflib.term.URIRef('class'), rdflib.term.URIRef('rdf:type')), (rdflib.term.URIRef('%(e_in_in)s'), '-%(e_in_in_to_e_in)s')])
('uri', [('x', '-%(e_in_to_e)s')])
{'x': [(rdflib.term.URIRef('class'), rdflib.term.URIRef('rdf:type')), (rdflib.term.URIRef('%(e_in_in)s'), '-%(e_in_in_to_e_in)s')], 'uri': [('x', '-%(e_in_to_e)s')]}
 SELECT DISTINCT ?uri WHERE { <%(e_in_in)s>  <%(e_in_in_to_e_in)s> ?x .  ?x <%(e_in_to_e)s> ?uri. ?x <rdf:type> <class>}
find(intersection(find(class, rdf:type), find(%(e_in_in)s, reverse(%(e_in_in_to_e_in)s))), reverse(%(e_in_to_e)s))
MATCH (x)-[:`rdf:type`]->(o0 {`uri`: $o0_uri}) MATCH (x)<-[:`%(e_in_in_to_e_in)s`]-(o0 {`uri`: $o0_uri}) MATCH (uri)<-[:`%(e_in_to_e)s`]-(o1 {`uri`: $o1_uri}) RETURN uri
OrderedDict([('o0_uri', rdflib.term.URIRef('%(e_in_in)s')), ('o1_uri', 'x')])


In [ ]:
import codecs, pickle
from dataclasses import dataclass, field
from typing import Dict

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

@dataclass
class Container:
    container: Dict[str,int] = field(default_factory=dict)
    counter: int = 0
        
    def get(self, element):
        if element in self.container:
            ret = self.container[element]
        else:
            ret = self.container[element] = self.counter
            self.counter += 1
        return ret
            
entities = Container()
predicates = Container()
triples = []


with codecs.open("/data/ssd1/dbpedia/triples.index.pickle", "rb") as f:
    (entities, predicates) = pickle.load(f)
    
with codecs.open("/data/ssd1/dbpedia/triples.pickle", "rb") as f:
    triples = pickle.load(f)
    
pred_id_uri = dict([(j,i) for i,j in predicates.container.items()])
ent_id_uri = dict([(j,i) for i,j in entities.container.items()])


In [49]:
from pypher import Pypher, Param
from dataclasses import dataclass

@dataclass
class Grammar:
    p: Pypher

    def blank_node(self, var_in: str):
        self.p.node(var_in)

    def uri_node(self, var_in: str, e: str):
        param = Param(f'{var_in}_uri', e)
        self.p.node(var_in, uri=param)
    
    def find(self, var_out: str, var_in, e: str, r: str):
        self.p.Match.node(var_out)
        
        if r.startswith("-"):
            self.p.rel_in(labels=r[1:])
        else:
            if(r.startswith("+")):
                r = r[1:]
            self.p.rel_out(labels=r)
        if e:
            self.uri_node(var_in, e)
        else:
            self.blank_node(var_in)
        
    def get_return(self, var):
        getattr(self.p.RETURN, var)
        return self.p
    
    # def find(self, x: str, r: str):
    #     self.p.Match.node('x')

g = Grammar(Pypher())
g.find('x', "tvshow", "http://dbpedia.org/ontology/TelevisionShow", f"rRT")
g.find('x', 'y', None, f"R")
g.find('y', "docextra", 'http://dbpedia.org/resource/Doctor_Who_Extra', f"rSW")
cypher = g.get_return('x')
print(cypher)
print(cypher.bound_params)

# g = Grammar(Pypher())
# cypher = g.find("http://dbpedia.org/ontology/TelevisionShow", f"-r2")
# print(cypher)
# print(cypher.bound_params)
# print(len(pred_id_uri))
# print(node("http://blah"))

# MATCH (e:Entity {uri:"http://dbpedia.org/ontology/TelevisionShow"})<-[r {uri:"http://www.w3.org/1999/02/22-rdf-syntax-ns#type"}]-(x) RETURN r,x

# MATCH (s)-[r]->(e:Entity {uri: 'http://dbpedia.org/resource/India'}) RETURN s,r


MATCH (x)-[:`rRT`]->(tvshow {`uri`: $tvshow_uri}) MATCH (x)-[:`R`]->(y) MATCH (y)-[:`rSW`]->(docextra {`uri`: $docextra_uri}) RETURN x
OrderedDict([('docextra_uri', 'http://dbpedia.org/resource/Doctor_Who_Extra'), ('tvshow_uri', 'http://dbpedia.org/ontology/TelevisionShow')])


In [36]:
    g = Grammar(Pypher())
    c = g.find('http://dbpedia.org/ontology/TelevisionShow',
             f"r2")
    print(c)


MATCH (out)-[:`r2`]->(e {`euri`: $euri}) RETURN out


In [52]:
from neo4j import GraphDatabase

# driver = GraphDatabase.driver("bolt://192.168.178.150:7687", auth=('neo4j', 'password'))

g = Grammar(Pypher())
g.find('x', "tvshow", "http://dbpedia.org/ontology/TelevisionShow", f"r2")
g.find('x', 'y', None, f"1297")
g.find('y', "docextra", 'http://dbpedia.org/resource/Doctor_Who_Extra', f"r1371")
c = g.get_return('x')
print(c)
print(c.bound_params)

def query(tx):
    # g = Grammar(Pypher())
    # c = g.find("http://dbpedia.org/ontology/Work", f":r{predicates.container['http://www.w3.org/1999/02/22-rdf-syntax-ns#type']}")
    c.LIMIT(10)
    print(c)
    result = tx.run(str(c).replace("e()", "e"), **c.bound_params)
    return result.single()[0]

with driver.session() as session:
    out = session.write_transaction(query)
    print(out)


MATCH (x)-[:`r2`]->(tvshow {`uri`: $tvshow_uri}) MATCH (x)-[:`1297`]->(y) MATCH (y)-[:`r1371`]->(docextra {`uri`: $docextra_uri}) RETURN x
OrderedDict([('docextra_uri', 'http://dbpedia.org/resource/Doctor_Who_Extra'), ('tvshow_uri', 'http://dbpedia.org/ontology/TelevisionShow')])
MATCH (x)-[:`r2`]->(tvshow {`uri`: $tvshow_uri}) MATCH (x)-[:`1297`]->(y) MATCH (y)-[:`r1371`]->(docextra {`uri`: $docextra_uri}) RETURN x LIMIT 10


TypeError: 'NoneType' object is not subscriptable